导入包

In [1]:
import torch as th 
th.backends.cudnn.benchmark = True

In [2]:
from mainNet import Decoder_Train_Net

train_net = Decoder_Train_Net(get_layer_output= False, 
                              init= True,
                              Normalized= False).cuda()

加载数据

In [3]:
from dataloader import ImageLoder
from torch.utils.data import DataLoader

dataset = DataLoader(dataset= ImageLoder(r"./data\train2014", 50, True, preload= True), 
                     batch_size= 1, 
                     shuffle= True, 
                     num_workers= 4,
                     pin_memory= True, 
                     prefetch_factor= 2,
                
)

In [ ]:
for i in dataset: 
    print(i)

加载优化器、训练调整器、损失函数

In [4]:
optimizer = th.optim.Adam(train_net.decoder.parameters(), lr= 0.001)

In [5]:
scheduler = th.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode= 'min', 
                                                    factor= 0.5, patience= 5, 
                                                    verbose= True, min_lr= 1e-4)

In [6]:
from lossFunction import Decoder_Train_Loss

loss_fn = Decoder_Train_Loss().cuda()

设置训练轮次，加载预训练解码器

In [7]:
epochs = 20

In [8]:
now_epoch= 1

In [ ]:
train_net.decoder.load_state_dict(th.load(r"pretrained_models\decoder1.4.pth"))

In [ ]:
# from torch.utils.tensorboard import SummaryWriter
# writer = SummaryWriter(utils.get_logPath(r"./logs"))

训练

In [9]:
sum = dataset.__len__()

In [10]:
train_net.train()
for _ in range(epochs): 
    loss_all = 0 
    print("epoch: {:d}".format(now_epoch))
    for img in dataset:
        img = img.cuda()
        output = train_net(img)
        
        # loss = loss_fn(train_net.dlist ,train_net.elist)
        loss = loss_fn(output, img, True)
        
        loss_all += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    scheduler.step(loss_all)
    # writer.add_scalar("loss", loss_all, now_epoch)
    print("loss: ", loss_all / sum)
    now_epoch += 1

epoch: 1
loss:  nan
epoch: 2
loss:  nan
epoch: 3


KeyboardInterrupt: 

梯度检查

In [11]:
for name, parms in train_net.decoder.named_parameters():
	print('-->name:', name, '--weight', th.mean(parms.data), ' -->grad_value:', th.mean(parms.grad))

-->name: 0.1.weight --weight tensor(nan, device='cuda:0')  -->grad_value: tensor(nan, device='cuda:0')
-->name: 0.1.bias --weight tensor(nan, device='cuda:0')  -->grad_value: tensor(-inf, device='cuda:0')
-->name: 0.5.weight --weight tensor(nan, device='cuda:0')  -->grad_value: tensor(nan, device='cuda:0')
-->name: 0.5.bias --weight tensor(nan, device='cuda:0')  -->grad_value: tensor(-inf, device='cuda:0')
-->name: 0.8.weight --weight tensor(nan, device='cuda:0')  -->grad_value: tensor(nan, device='cuda:0')
-->name: 0.8.bias --weight tensor(nan, device='cuda:0')  -->grad_value: tensor(-inf, device='cuda:0')
-->name: 0.11.weight --weight tensor(nan, device='cuda:0')  -->grad_value: tensor(nan, device='cuda:0')
-->name: 0.11.bias --weight tensor(nan, device='cuda:0')  -->grad_value: tensor(-inf, device='cuda:0')
-->name: 1.1.weight --weight tensor(nan, device='cuda:0')  -->grad_value: tensor(nan, device='cuda:0')
-->name: 1.1.bias --weight tensor(nan, device='cuda:0')  -->grad_value: ten

模型保存

In [ ]:
th.save(train_net.decoder.state_dict(), r"./pretrained_models/decoder2_0.1.pth")

测试

In [ ]:
import utils
import numpy as np
img = utils.get_pilimg(r"data\test\Lenna.png")
img_tensor = utils.pilimg2tensor(img, cuda= True)

In [ ]:
train_net.eval()
output = train_net(img_tensor)
r_img = utils.tensor2img(output)
utils.img_display(r_img, 'test')

In [ ]:
random_img = np.random.random((1, 3, 512, 512)).astype(np.float32)
test_img = th.from_numpy(random_img).cuda()

In [ ]:
test_output = train_net(test_img)

In [ ]:
test_result = utils.tensor2img(test_output)
utils.img_display(test_result, 'test_random')